# Database Normalization

## First normal form (1NF)

Each table has a primary key: minimal set of attributes which can uniquely identify a record.<br />
The values in each column of a table are atomic (No multi-value attributes allowed).<br />
There are no repeating groups: two columns do not store similar information in the same table.<br />

In [3]:
import pandas as pd

In [4]:
treatments_clean = pd.read_csv('treatments_clean.csv')

In [5]:
treatments_clean

,hba1c_start,hba1c_end,hba1c_change,treatment,dose,dose_start,dose_end,adverse_reaction,patient_id
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,"headache, nausea",225
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,"hypoglycemia, body pain",242
2,7.65,7.27,0.38,auralin,37u - 42u,37,42,"throat irritation, cough, nausea",345
3,7.89,7.55,0.34,auralin,31u - 38u,31,38,"throat irritation, cough, nausea",276
4,7.76,7.37,0.39,auralin,30u - 36u,30,36,"headache, nausea",15
...,...,...,...,...,...,...,...,...,...
159,7.51,7.06,0.45,novodra,55u - 51u,55,51,nausea,153
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,"hypoglycemia, body pain",420
161,9.21,8.80,0.41,novodra,22u - 23u,22,23,injection site discomfort,336
162,7.96,7.51,0.45,novodra,28u - 26u,28,26,hypoglycemia,25


In [6]:
# Check the primary key is unique or not
treatments_clean['patient_id'].is_unique

True

In [7]:
treatments_clean_data = pd.read_csv('treatments_clean.csv')

In [8]:
treatments_clean_data

,hba1c_start,hba1c_end,hba1c_change,treatment,dose,dose_start,dose_end,adverse_reaction,patient_id
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,"headache, nausea",225
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,"hypoglycemia, body pain",242
2,7.65,7.27,0.38,auralin,37u - 42u,37,42,"throat irritation, cough, nausea",345
3,7.89,7.55,0.34,auralin,31u - 38u,31,38,"throat irritation, cough, nausea",276
4,7.76,7.37,0.39,auralin,30u - 36u,30,36,"headache, nausea",15
...,...,...,...,...,...,...,...,...,...
159,7.51,7.06,0.45,novodra,55u - 51u,55,51,nausea,153
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,"hypoglycemia, body pain",420
161,9.21,8.80,0.41,novodra,22u - 23u,22,23,injection site discomfort,336
162,7.96,7.51,0.45,novodra,28u - 26u,28,26,hypoglycemia,25


In [69]:
# Column 'adverse_reaction' has multi-value. Convert this column into another table.
df1 = treatments_clean_data['adverse_reaction'].str.split(',',expand = True).stack()
df1 = df1.reset_index(level = 1,drop=True).rename('adverse_reaction_temp')
df2 = treatments_clean_data
df2.join(df1)

,hba1c_start,hba1c_end,hba1c_change,treatment,dose,dose_start,dose_end,adverse_reaction,patient_id,adverse_reaction_temp
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,"headache, nausea",225,headache
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,"headache, nausea",225,nausea
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,"hypoglycemia, body pain",242,hypoglycemia
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,"hypoglycemia, body pain",242,body pain
2,7.65,7.27,0.38,auralin,37u - 42u,37,42,"throat irritation, cough, nausea",345,throat irritation
...,...,...,...,...,...,...,...,...,...,...
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,"hypoglycemia, body pain",420,hypoglycemia
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,"hypoglycemia, body pain",420,body pain
161,9.21,8.80,0.41,novodra,22u - 23u,22,23,injection site discomfort,336,injection site discomfort
162,7.96,7.51,0.45,novodra,28u - 26u,28,26,hypoglycemia,25,hypoglycemia


In [73]:
# Build adverse_reaction table
df3 = df2.join(df1)[['patient_id','adverse_reaction_temp']]
adverse_reaction_object = df3['adverse_reaction_temp']

# delete duplicate adverse_reaction
adverse_reaction_object = adverse_reaction_object.drop_duplicates(keep = 'first')

#reset index
adverse_reaction_object = adverse_reaction_object.reset_index(drop = True)

adverse_reaction_list = []
for i in adverse_reaction_object:
    adverse_reaction_list.append(i)

adverse_reaction_id = []
k = 1
for i in range(len(adverse_reaction_list)):
    adverse_reaction_id.append(k)
    k = k + 1

adverse_reaction_dict = {'adverse_reaction_id' : adverse_reaction_id,
                 'adverse_reaction_name' : adverse_reaction_list}

adverse_reaction = pd.DataFrame(adverse_reaction_dict)

adverse_reaction


,adverse_reaction_id,adverse_reaction_name
0,1,headache
1,2,nausea
2,3,hypoglycemia
3,4,body pain
4,5,throat irritation
5,6,cough
6,7,nausea
7,8,cold
8,9,injection site discomfort


In [74]:
# Check is there any missing value in  'adverse_reaction'
adverse_reaction.isna().any()

adverse_reaction_id      False
adverse_reaction_name    False
dtype: bool

In [75]:
# Check the primary key in 'adverse_reaction' is unique or not
adverse_reaction['adverse_reaction_id'].is_unique

True

In [82]:
# Create 'patient_adverse_reaction' table

id_list = []
for i in df3['patient_id']:
    id_list.append(i)

adverse_reaction_list = []
for i in df3['adverse_reaction_temp']:
    adverse_reaction_list.append(i)

patient_adverse_reaction_dict = {'patient_id':id_list,
                            'adverse_reaction_name':adverse_reaction_list}

patient_adverse_reaction_temp = pd.DataFrame(patient_adverse_reaction_dict)
patient_adverse_reaction = pd.merge(patient_adverse_reaction_temp,adverse_reaction,on='adverse_reaction_name',how='left',sort= False)
patient_adverse_reaction
patient_adverse_reaction = patient_adverse_reaction.drop(['adverse_reaction_name'],axis = 1)
patient_adverse_reaction

,patient_id,adverse_reaction_id
0,225,1
1,225,2
2,242,3
3,242,4
4,345,5
...,...,...
311,420,3
312,420,4
313,336,9
314,25,3


In [83]:
# Check is there any missing value in  'patient_adverse_reaction'
patient_adverse_reaction.isna().any()

patient_id             False
adverse_reaction_id    False
dtype: bool

In [87]:
# Check the primary kay in 'patient_adverse_reaction' is unique or not
judge = patient_adverse_reaction.duplicated()
k = 0
for i in judge:
    if i == False:
        k = k+1
    else:
        print('The primary key in patient_adverse_reaction is not unique.')
        break

if k == 316:
    print('The primary key in patient_adverse_reaction is unique.')

The primary key in patient_adverse_reaction is unique.


### Reformat 'patient' into 1NF

In [ ]:
patients_clean = pd.read_csv('patient_clean.csv')

In [89]:
patients_clean

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390,United States,1976-07-10,121.7,66,19.6,19517199170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,IL,61812,United States,1967-04-03,118.8,66,19.2,12175693204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467,United States,1980-02-19,177.8,71,24.8,14023636804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,07095,United States,1951-07-26,220.9,70,31.7,17326368246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303,United States,1928-02-18,192.3,72,26.1,13345157487,TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,498,male,Masataka,Murakami,1179 Patton Lane,Tulsa,OK,74116,United States,1937-08-19,155.1,72,21.0,19189849171,MasatakaMurakami@einrot.com
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,03852,United States,1959-04-10,181.1,72,24.6,12074770579,MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341,United States,1948-03-26,239.6,70,34.4,19282844492,RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110,United States,1971-01-13,171.2,67,26.8,18162236007,JinkedeKeizer@teleworm.us


In [94]:
# Save patients_clean
outputpath='C:/Users/Shinde/Desktop/DMDD/Assignment 4/Yelp-Restaurant-Database-MA/Upload/patient_clean.csv'
patients_clean.to_csv(outputpath,sep=',',index=False,encoding = 'utf-8',header=True)

In [139]:
patient_clean_data = pd.read_csv('patient_clean.csv')

In [140]:
# Check the primary key is unique or not
patient_clean_data['patient_id'].is_unique

False

In [141]:
# delete duplicate 'patient_id'
patient_clean_data = patient_clean_data.drop_duplicates(subset = 'patient_id',keep = 'first')
#reset index
patient_clean_data = patient_clean_data.reset_index(drop = True)
# check 'patient_id' is unique or nor again
patient_clean_data['patient_id'].is_unique

True

### Show all tables in 1NF

In [133]:
# Show head of 'adverse_reaction'
adverse_reaction.head()

,adverse_reaction_id,adverse_reaction_name
0,1,headache
1,2,nausea
2,3,hypoglycemia
3,4,body pain
4,5,throat irritation


In [134]:
# Show head of 'patient_adverse_reaction'
patient_adverse_reaction.head()

,patient_id,adverse_reaction_id
0,225,1
1,225,2
2,242,3
3,242,4
4,345,5


In [135]:
# Show head of 'patient_clean_data'
patient_clean_data.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390,United States,10-07-1976,121.7,66,19.6,1.951720e+10,ZoeWellish@superrito.com
1,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467,United States,19-02-1980,177.8,71,24.8,1.402364e+10,JaeMDebord@gustr.com
2,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,70955,United States,26-07-1951,220.9,70,31.7,1.732637e+10,PhanBaLiem@jourrapide.com
3,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303,United States,18-02-1928,192.3,72,26.1,1.334516e+10,TimNeudorf@cuvox.de
4,6,male,Rafael,Costa,1140 Willis Avenue,Daytona Beach,FL,32114,United States,31-08-1931,183.9,70,26.4,1.386335e+10,RafaelCardosoCosta@gustr.com


## Second normal form (2NF)

All requirements for 1st NF must be met.<br />
No partial dependencies.<br />
No calculated data.<br />

### Reformat 'treatment' into 2NF

In [108]:
treatments_clean_data

,hba1c_start,hba1c_end,hba1c_change,treatment,dose,dose_start,dose_end,adverse_reaction,patient_id
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,"headache, nausea",225
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,"hypoglycemia, body pain",242
2,7.65,7.27,0.38,auralin,37u - 42u,37,42,"throat irritation, cough, nausea",345
3,7.89,7.55,0.34,auralin,31u - 38u,31,38,"throat irritation, cough, nausea",276
4,7.76,7.37,0.39,auralin,30u - 36u,30,36,"headache, nausea",15
...,...,...,...,...,...,...,...,...,...
159,7.51,7.06,0.45,novodra,55u - 51u,55,51,nausea,153
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,"hypoglycemia, body pain",420
161,9.21,8.80,0.41,novodra,22u - 23u,22,23,injection site discomfort,336
162,7.96,7.51,0.45,novodra,28u - 26u,28,26,hypoglycemia,25


In [120]:
treatments_clean_data = pd.read_csv('treatments_clean_data.csv')
treatments_clean_data

,hba1c_start,hba1c_end,hba1c_change,treatment,dose,dose_start,dose_end,adverse_reaction,patient_id
0,7.63,7.20,0.43,auralin,41u - 48u,41,48,cough,225
1,7.97,7.62,0.35,auralin,33u - 36u,33,36,body pain,242
2,7.65,7.27,0.38,auralin,37u - 42u,37,42,throat irritation,345
3,7.89,7.55,0.34,auralin,31u - 38u,31,38,hypoglycemia,276
4,7.76,7.37,0.39,auralin,30u - 36u,30,36,nausea,15
...,...,...,...,...,...,...,...,...,...
159,7.51,7.06,0.45,novodra,55u - 51u,55,51,nausea,153
160,7.67,7.30,0.37,novodra,26u - 23u,26,23,body pain,420
161,9.21,8.80,0.41,novodra,22u - 23u,22,23,injection site discomfort,336
162,7.96,7.51,0.45,novodra,28u - 26u,28,26,hypoglycemia,25


In [121]:
# 'treatment', and 'adverse_reaction' is dependent on 'patient_id', so take these columns out to form a new table 'treatment_reaction'.
treatment_reaction = treatments_clean_data[['patient_id','treatment','adverse_reaction']]
treatment_reaction.head()

,patient_id,treatment,adverse_reaction
0,225,auralin,cough
1,242,auralin,body pain
2,345,auralin,throat irritation
3,276,auralin,hypoglycemia
4,15,auralin,nausea


### Show all tables in 2NF

In [136]:
# Show patient_clean_data
patient_clean_data.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390,United States,10-07-1976,121.7,66,19.6,1.951720e+10,ZoeWellish@superrito.com
1,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467,United States,19-02-1980,177.8,71,24.8,1.402364e+10,JaeMDebord@gustr.com
2,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,70955,United States,26-07-1951,220.9,70,31.7,1.732637e+10,PhanBaLiem@jourrapide.com
3,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303,United States,18-02-1928,192.3,72,26.1,1.334516e+10,TimNeudorf@cuvox.de
4,6,male,Rafael,Costa,1140 Willis Avenue,Daytona Beach,FL,32114,United States,31-08-1931,183.9,70,26.4,1.386335e+10,RafaelCardosoCosta@gustr.com


In [123]:
# Show head of 'patient_adverse_reaction'
patient_adverse_reaction.head()

,patient_id,adverse_reaction_id
0,225,1
1,225,2
2,242,3
3,242,4
4,345,5


In [124]:
# Show head of 'adverse_reaction'
adverse_reaction.head()

,adverse_reaction_id,adverse_reaction_name
0,1,headache
1,2,nausea
2,3,hypoglycemia
3,4,body pain
4,5,throat irritation


In [125]:
# Show 'treatments_clean_data'
del treatments_clean_data['treatment']
del treatments_clean_data['adverse_reaction']
treatments_clean_data.head()

,hba1c_start,hba1c_end,hba1c_change,dose,dose_start,dose_end,patient_id
0,7.63,7.20,0.43,41u - 48u,41,48,225
1,7.97,7.62,0.35,33u - 36u,33,36,242
2,7.65,7.27,0.38,37u - 42u,37,42,345
3,7.89,7.55,0.34,31u - 38u,31,38,276
4,7.76,7.37,0.39,30u - 36u,30,36,15


In [126]:
# Show 'treatment_reaction'
treatment_reaction.head()

,patient_id,treatment,adverse_reaction
0,225,auralin,cough
1,242,auralin,body pain
2,345,auralin,throat irritation
3,276,auralin,hypoglycemia
4,15,auralin,nausea


### Third normal form (3NF)
All requirements for 2nd NF must be met.<br />
Eliminate fields that do not directly depend on the primary key; that is no transitive dependencies.<br />

### Reformat 'patient' into 3NF

In [142]:
# In table 'patient', there is an indirect relationship between 'patient_id' and 'city' : 'patient_id' -> 'zip_code', 'zip_code' -> 'city'. 
# So, the table 'restaurant' is not in the 3NF.

city = patient_clean_data[['zip_code','city']]
city = city.drop_duplicates(subset = 'zip_code',keep='first')
city.head()

,zip_code,city
0,92390,Rancho California
1,68467,York
2,70955,Woodbridge
3,36303,Dothan
4,32114,Daytona Beach


In [143]:
# Check if there is any missing value in 'city'
city.isnull().any()

zip_code    False
city        False
dtype: bool

In [144]:
# Check if the primary key is unique or not
city['zip_code'].is_unique

True

### Show all table in 3NF

In [145]:
# Save 'patient'
del patient_clean_data['city']
outputpath='C:/Users/Shinde/Desktop/DMDD/Assignment 4/Yelp-Restaurant-Database-MA/Upload/patient_clean_1.csv'
patient_clean_data.to_csv(outputpath,sep=',',index=False,encoding = 'utf-8',header=True)

In [146]:
# Show 'patient'
patient_clean_data.head()

,patient_id,assigned_sex,given_name,surname,address,state,zip_code,country,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,CA,92390,United States,10-07-1976,121.7,66,19.6,1.951720e+10,ZoeWellish@superrito.com
1,3,male,Jae,Debord,1493 Poling Farm Road,NE,68467,United States,19-02-1980,177.8,71,24.8,1.402364e+10,JaeMDebord@gustr.com
2,4,male,Liêm,Phan,2335 Webster Street,NJ,70955,United States,26-07-1951,220.9,70,31.7,1.732637e+10,PhanBaLiem@jourrapide.com
3,5,male,Tim,Neudorf,1428 Turkey Pen Lane,AL,36303,United States,18-02-1928,192.3,72,26.1,1.334516e+10,TimNeudorf@cuvox.de
4,6,male,Rafael,Costa,1140 Willis Avenue,FL,32114,United States,31-08-1931,183.9,70,26.4,1.386335e+10,RafaelCardosoCosta@gustr.com


In [147]:
# Show head of 'adverse_reaction'
adverse_reaction.head()

,adverse_reaction_id,adverse_reaction_name
0,1,headache
1,2,nausea
2,3,hypoglycemia
3,4,body pain
4,5,throat irritation


In [148]:
# Show head of 'patient_adverse_reaction'
patient_adverse_reaction.head()

,patient_id,adverse_reaction_id
0,225,1
1,225,2
2,242,3
3,242,4
4,345,5


In [151]:
# Show 'treatments_clean_data'
treatments_clean_data.head()

,hba1c_start,hba1c_end,hba1c_change,dose,dose_start,dose_end,patient_id
0,7.63,7.20,0.43,41u - 48u,41,48,225
1,7.97,7.62,0.35,33u - 36u,33,36,242
2,7.65,7.27,0.38,37u - 42u,37,42,345
3,7.89,7.55,0.34,31u - 38u,31,38,276
4,7.76,7.37,0.39,30u - 36u,30,36,15


In [152]:
# Show 'treatment_reaction'
treatment_reaction.head()

,patient_id,treatment,adverse_reaction
0,225,auralin,cough
1,242,auralin,body pain
2,345,auralin,throat irritation
3,276,auralin,hypoglycemia
4,15,auralin,nausea


In [153]:
# Show 'city'
city.head()

,zip_code,city
0,92390,Rancho California
1,68467,York
2,70955,Woodbridge
3,36303,Dothan
4,32114,Daytona Beach
